<a href="https://colab.research.google.com/github/alevinas/minecraftImagesRNA/blob/main/minecraftImagesRecognize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Own dataset with minecraft mobs images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
